<a href="https://colab.research.google.com/github/reinaldobarberan/llms/blob/main/Week_3_Day_5_Meeting_Minutes_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crear actas de reuniones a partir de un archivo de audio

Descargué algunas actas de reuniones del Ayuntamiento de Denver y seleccioné una parte para transcribirla. Puede descargarla aquí: https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

Si prefiere trabajar con los datos originales, el conjunto de datos de HuggingFace está [aquí](https://huggingface.co/datasets/huuuyeah/meetingbank) y el audio se puede descargar [aquí](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

El objetivo de este producto es usar el audio para generar actas de reuniones, incluyendo las acciones.

Para este proyecto, puede usar las actas de reuniones de Denver o grabar algo propio.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Nuevamente, tenga en cuenta: Dos consejos importantes para usar Colab:

**Consejo 1:**

Al principio de cada Colab se instalan algunas instalaciones de pip. Es posible que reciba errores de pip al ejecutarlo, como:

> gcsfs 2025.3.2 requiere fsspec==2025.3.2, pero tiene fsspec 2025.3.0, que es incompatible.

Estos errores de compatibilidad de pip se pueden ignorar sin problemas; y aunque es tentador intentar solucionarlos cambiando los números de versión, ¡eso en realidad causará problemas graves!

**Consejo 2:**

Durante la ejecución de un Colab, podría recibir un error como este:

> Error en tiempo de ejecución: Se requiere CUDA, pero no está disponible para bitsandbytes. Considere instalar [...]

¡Este es un mensaje de error muy engañoso! Por favor, no intentes cambiar las versiones de los paquetes...

Esto ocurre porque Google ha cambiado el entorno de ejecución de Colab, quizás porque Google Colab estaba demasiado ocupado. La solución es:

1. Menú del kernel >> Desconectar y eliminar el entorno de ejecución
2. Recargar Colab desde cero y menú Editar >> Borrar todas las salidas
3. Conectarse a un nuevo T4 con el botón de la esquina superior derecha
4. Seleccionar "Ver recursos" en el menú de la esquina superior derecha para confirmar que se dispone de una GPU
5. Volver a ejecutar las celdas en Colab, de arriba a abajo, empezando por las instalaciones de pip

Todo debería funcionar correctamente; si no, ¡consúltame!

In [1]:
!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [8]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

drive.mount("/content/drive")
audio_filename = "/content/denver_extract.mp3"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Descargar denver_extract.mp3

Puedes usar el mismo archivo que yo (el extracto de las actas del ayuntamiento de Denver) o crear uno propio.

Si quieres usar el mismo archivo que yo, descarga mi extracto aquí y guárdalo en tu Google Drive:

https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

In [9]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text
# If you'd prefer to use an Open Source model, class student Youssef has contributed an open source version
# which I've added to the bottom of this colab

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

In [11]:
!pip install torch transformers accelerate sentencepiece datasets soundfile

In [17]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Transcripción con Whisper
asr = pipeline("automatic-speech-recognition", model="openai/whisper-small")
transcription = asr(audio_filename, return_timestamps=True)["text"]


Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was us

In [18]:
transcription

" and kind of the confluence of this whole idea of the confluence week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let you see the creation of the logo here. And then, yes, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these Indigenous events and things that are happening around Denver so that we can and kind of bring more people together and kind of share this whole idea of indigenous people's day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to th

In [19]:
system_message = "Eres un asistente que produce actas de reuniones a partir de transcripciones, con resumen, puntos clave de discusión, conclusiones y elementos de acción con los propietarios, en formato Markdown."
user_prompt = f"A continuación se presenta un extracto de la transcripción de una reunión del consejo de Denver. Por favor, redacte el acta en formato Markdown, incluyendo un resumen con los asistentes, el lugar y la fecha; los puntos de discusión; las conclusiones; y las acciones a tomar con los propietarios.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [12]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [20]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Eres un asistente que produce actas de reuniones a partir de transcripciones, con resumen, puntos clave de discusión, conclusiones y elementos de acción con los propietarios, en formato Markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

A continuación se presenta un extracto de la transcripción de una reunión del consejo de Denver. Por favor, redacte el acta en formato Markdown, incluyendo un resumen con los asistentes, el lugar y la fecha; los puntos de discusión; las conclusiones; y las acciones a tomar con los propietarios.
 and kind of the confluence of this whole idea of the confluence week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let

In [21]:
response = tokenizer.decode(outputs[0])

In [22]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Eres un asistente que produce actas de reuniones a partir de transcripciones, con resumen, puntos clave de discusión, conclusiones y elementos de acción con los propietarios, en formato Markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

A continuación se presenta un extracto de la transcripción de una reunión del consejo de Denver. Por favor, redacte el acta en formato Markdown, incluyendo un resumen con los asistentes, el lugar y la fecha; los puntos de discusión; las conclusiones; y las acciones a tomar con los propietarios.
 and kind of the confluence of this whole idea of the confluence week, the merging of two rivers. And as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason that the back of the logo is considered water. So let you see the creation of the logo here. And then, yes, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our Confluence Week is basically highlighting all of these Indigenous events and things that are happening around Denver so that we can and kind of bring more people together and kind of share this whole idea of indigenous people's day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America and to the republic for which it stands. All right, thank you councilman Lopez madam secretary roll call black Clark here Espanosa here Flynn Gilmore here Here cashman here can each here Lopez here new here or taiga Susman mr. President Here 11 present 11 members present. We do have a quorum approval of the minutes. Are there any corrections to the minutes of October 2nd? Seeing none minutes of October 2nd stand approve council announcements. Are there any announcements by members of council? Councilman Clark. Thank you, Mr. President. I just wanted to invite everyone down to the first ever Halloween parade on Broadway in Lucky District 7. It will happen on Saturday, October 21st at 6 o'clock p.m. It will move along Broadway from 3rd to Alameda. It's going to be a fun family friendly event. Everyone's invited to come down, wear a costume. There There'll be candy for the kids and there are Tiki zombies and 29 herses and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us October 21st at six o'clock for the Broadway Halloween. Brad, thank you, Mr. President. All right, thank you, Councilman Clark. I will be there. All right, presentations. Madam Secretary, do we have any presentations? None, Mr. President. communications do we have any communications? None, Mr. President. We do have one proclamation this evening, proclamation 1127, an observance of the annual Indigenous Peoples Day in the City and County of Denver. Councilman Low players, will you please read it? Thank you, Mr. President. Pride. Proclamation number 17, well, let me just say this differently. Proclamation number 1127 series of 2017, and observance of the second annual Indigenous Peoples' Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the Indigenous Peoples have lived and flourished on the lands known as the Amedica since time immemorial. And that Denver and the surrounding communities are built upon the ancestral homelands of numerous Indigenous tribes, which include the southern Yute Mountain, Yute tribes of Colorado, and whereas the tribal homelands and seasonal encampments of the Arapaho and Sham people along the banks of the Cherry Creek and South Platte River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes in the city and county of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas, on October 3rd, 2016, the City and County of Denver unanimously passed Council Bill 801, series of 2016, officially designating the second Monday of October of each year, as Indigenous Peoples Day in Denver, Colorado. And whereas, the Council of the City and County of Denver continues to recognize and value the vast contributions contributions made to the community through indigenous peoples knowledge, science, philosophy, arts, and culture, and through these contributions the city of Denver has developed and thrived. Whereas the indigenous community, especially youth, have made great efforts this year to draw attention to the contributions of indigenous people including Confluence Week, drawing record We are detentives to a national indigenous youth leadership conference, leading conversations on inclusion with their peers, and supporting increased indigenous youth participation in science and engineering. Now therefore, be it proclaimed by the Council of the City and County of Denver, Section 1, that the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of indigenous people to our history. our past, present, and future. And continues to promote the education of the Denver community about these historical and contemporary contributions of indigenous people. Section two, at the City and County of Denver, Colorado does hereby observe October 9th, 2017 as indigenous people's day. Section three, at the clerk of the city and county of Denver shall attest and affix the seal of the city and county of Denver to this proclamation. And that a copy be transmitted, excuse me, to the Denver American Indian Commission, the city and county of Denver School District number one, and the Colorado Commission on Indian Affairs. Thank you, Councilman Lopez, your motion to adopt. President, I move that proclamation number 1127, series of 2017 be adopted. All right, it has been moved and seconded. Comes with members of council, Councilor Lopez. Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples Day in Denver officially for the second time. It is, it's always awesome to be able to see not just this proclamation come by my desk, but to see so many different people from our community in our council chambers. It was a very beautiful piece of artwork that you presented to us earlier. And it is exactly the spirit that we drafted this proclamation and this actual, the ordinance that created Indigenous Peoples Day when we sat down and wrote it. And as a community, we couldn't think of anything else to begin except for the confluence of the two rivers. And those confluence of the two rivers created such a great city. And we live in such an amazing city. And we're all proud of it. And sometimes we, and a lot of people from all over the country or all over the world are proud of it. And sometimes a little too proud of it, is just telling them to go back home. But I'm kidding when I say that. But the really nice thing about this is that we are celebrating Indigenous people's day out of pride for who we are, who we are as a city, and the contributions of Indigenous people to the city, not out of spite, not out of a replacement of one culture over the other, or out of contempt, or disrespect. You know, I think of a quote that Cesar Chavez made very popular, and it stuck with me for a very long time. And anytime I have the opportunity, I speak in front of children, and especially children in our community that, often second guess themselves on where they're coming from, who they are. And I always say that it's very important to be proud of who you're from. And the quote that I use from Cesar Chavez is, pride in one's own culture does not require contempt or disrespect of another, right? And that's very important. It's very important for us to recognize that. No matter who we are, where we come from in this society, that your pride in your own culture doesn't require, should not require the contempt or disrespect of another. And man, what a year to be, for that to just sit on our shoulders for a while, for us to think about, right? And so I wanted to just to thank you all, I think the commission, there's gonna be a couple individuals that are gonna come speak, thank you for your art, your lovely artwork for us to see what's in your heart and what now has become, probably is gonna be a very important symbol for the community. And also just for the work, the daily work every single day, we still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners, right, in poverty. without access to services, right? Without access to sobriety or even housing or jobs. And what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores, right? So we have a lot of work to do. And these kind of proclamations in this day is not a day off, it's a day on in Denver, right? right and addressing those critical issues. So I know that my colleagues are very supportive. I'm gonna ask you to support this proclamation as I know you always have done in the past. I'm very proud of today. Oh, and we made Time Magazine and Newsweek once again today as being a leader in terms of the cities that are celebrating Indigenous Peoples Day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Council Martega. Mr. President, I want to ask that my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the commission on Indian affairs was created and had the benefit of being able to go down to the four corners for a peace treaty signing ceremony between the youths and the Comanches that had been sort of at odds with each other for about 100 years. And just being able to participate in that powwow was pretty awesome. So and for those of you who continue to participate in the annual powwow, It's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on, as well as the native language, from generation to generation is just so incredible, because in so many cultures, people have come here and assimilated to the norms here, and they lose their language. and lose a lot of the culture. And in the native community, that hasn't happened. That has, you know, commitment to just passing that on from generation to generation is so important. And so I'm happy to be a co-sponsor of this tonight. Thank you. All right, thank you, Councilwoman Artega. Councilwoman Keneige. Thank you very much. And I also want to thank my colleague for bringing this forward. And I just wanted to say a word to the artist about how beautiful and moving I thought this logo was and your description of it. And I think one of the things that is clear is, the words sometimes don't convey the power of imagery or music or the other pieces that make up culture. And so I think the art is so important. And when you talked about water, I was also thinking about land. And I guess I just wanted to say thank you, Many of the Native American peoples of Colorado have been at the forefront, or actually nationally, of defending some of the public lands that have been protected over the last few years that are under attack right now. And there are places that the communities have fought to protect, but that everyone gets to enjoy. And so I just think that it's an example of where cultural preservation intersects with environmental protection, with recreation, and all of the other ways that public lands so important and so I think I just wanted to say thank you for that because I think we have some very sacred places in our country that are at risk right now and So as we celebrate I appreciate that there's still a piece of resistance in here And I think that I just want to mention a solidarity and I mentioned a feeling of solidarity with that resistance So thank you and Happy Confluence Week Thank you councilman kinech and See no other comments. I'll just say a couple and In a time of such divisive Ugliness and just despicable behavior from our leadership The reason I'm so supportive of indigenous people's days because it means inclusivity it means respecting all respecting those who have been silenced On purpose for a long time and whose history has not been told and so we celebrate inclusivity in the face of such evil times, honestly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Acta de la Reunión del Consejo de Denver**
**Fecha:** 9 de octubre de 2017
**Lugar:** Sala de Consejo del Ayuntamiento de Denver
**Asistentes:**
- Councilman Lopez
- Councilman Clark
- Councilwoman Artega
- Councilwoman Keneige
- Otros miembros del Consejo

**Resumen:**

En la reunión del Consejo de Denver del 9 de octubre de 2017, se discutió la proclamación del Día de los Pueblos Indígenas en la ciudad y condado de Denver. El Consejo aprobó la proclamación, que celebra y honra las contribuciones culturales y fundacionales de los pueblos indígenas a la historia de la ciudad.

**Puntos de discusión:**

1. Presentación del logo del Día de los Pueblos Indígenas, que representa la confluencia de los ríos Cherry Creek y South Platte River.
2. Lectura de la proclamación del Día de los Pueblos Indígenas, que reconoce las contribuciones de los pueblos indígenas a la ciudad y condado de Denver.
3. Discusión sobre la importancia de la inclusión y la respeto a la cultura y la historia de los pueblos indígenas.

**Conclusiones:**

1. El Consejo aprobó la proclamación del Día de los Pueblos Indígenas en la ciudad y condado de Denver.
2. El logo del Día de los Pueblos Indígenas se convirtió en un símbolo importante de la ciudad y condado de Denver.
3. El Consejo reconoció la importancia de la inclusión y la respeto a la cultura y la historia de los pueblos indígenas.

**Acciones a tomar:**

1. El Consejo se comprometió a promover la educación y la conciencia sobre la historia y la cultura de los pueblos indígenas en la ciudad y condado de Denver.
2. El Consejo se comprometió a trabajar en la protección y conservación de los lugares sagrados y culturales de los pueblos indígenas en la ciudad y condado de Denver.
3. El Consejo se comprometió a apoyar a la comunidad indígena en la ciudad y condado de Denver en sus esfuerzos para preservar y promover su cultura y historia.<|eot_id|>

# Contribución del estudiante

El estudiante Emad S. creó esta potente variación que usa `TextIteratorStreamer` para transmitir los resultados a una interfaz de usuario de Gradio y aprovecha los subprocesos en segundo plano para mejorar el rendimiento. La comparto aquí si quieres echar un vistazo a este trabajo muy interesante. ¡Gracias, Emad!

https://colab.research.google.com/drive/1Ja5zyniyJo5y8s1LKeCTSkB2xyDPOt6D

## Implementación alternativa

Youssef, alumno de la clase, contribuyó con esta variante, en la que utilizamos un modelo de código abierto para transcribir el audio de la reunión.

¡Gracias, Youssef!

In [23]:
AUDIO_MODEL = "openai/whisper-medium"
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device='cuda',
)

NameError: name 'AutoModelForSpeechSeq2Seq' is not defined

In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text
result = pipe(audio_filename)

In [ ]:
transcription = result["text"]
print(transcription)